# Consulta e Preparação de Dados
### CMC-13 - Introdução à Ciência de Dados

* Emanuel Reinaldo Gomes Bezerra
* Pedro Pinheiro Borges

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

### Leitura dos Arquivos de Dados

In [2]:
books = pd.read_csv("datasets/books.csv", delimiter=';')
users = pd.read_csv("datasets/users.csv", delimiter=';')
ratings = pd.read_csv("datasets/ratings.csv", delimiter=';')

In [3]:
books.columns, users.columns, ratings.columns

(Index(['isbn', 'book_title', 'book_author', 'year_of_publication', 'publisher',
        'img_l', 'Language', 'Category'],
       dtype='object'),
 Index(['user_id', 'age', 'city', 'state', 'country'], dtype='object'),
 Index(['isbn', 'user_id', 'rating'], dtype='object'))

### Criação do Dataset principal

In [4]:
ratings_users = pd.merge(ratings, users, on="user_id", how="inner")
dataset = pd.merge(ratings_users, books, on="isbn", how="inner").dropna()

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19720 entries, 0 to 20701
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   isbn                 19720 non-null  object 
 1   user_id              19720 non-null  int64  
 2   rating               19720 non-null  int64  
 3   age                  19720 non-null  float64
 4   city                 19720 non-null  object 
 5   state                19720 non-null  object 
 6   country              19720 non-null  object 
 7   book_title           19720 non-null  object 
 8   book_author          19720 non-null  object 
 9   year_of_publication  19720 non-null  float64
 10  publisher            19720 non-null  object 
 11  img_l                19720 non-null  object 
 12  Language             19720 non-null  object 
 13  Category             19720 non-null  object 
dtypes: float64(2), int64(2), object(10)
memory usage: 2.3+ MB


#### Remoção de Colunas
Removemos ``isbn`` e ``user_id`` pois são identificadores e não possuem informação atrelada a eles.

Removemos ``img_l`` porque é apenas um link para a imagem da capa do livro, sem informação atrelada ao livro.

Removemos ``Language`` porque apresenta dois valores únicos (``en`` e ``9``) com pouca ou nenhuma informação atrelada, pois ``9`` não é um indicativo adequado para idioma.

In [6]:
dataset.Language.unique()

array(['en', '9'], dtype=object)

In [7]:
dataset = dataset.drop(columns=["isbn", "user_id", "img_l", "Language"])

Removemos as colunas ``city``, ``state``, ``country``, porque não estamos interessados em utilizar localização para determinar a avaliação de uma pessoa em um dado livro.

In [8]:
print(len(dataset.country.unique()), len(dataset.state.unique()), len(dataset.city.unique()))

147 652 5616


In [9]:
dataset = dataset.drop(columns=["country", "state", "city"])

Análise das colunas ``book_author``, ``book_title``, ``Category`` e ``publisher``
* Existem algumas linhas cujo ``Category`` é ``9``. Iremos remover esses exemplares porque isto aparenta ser problema de qualidade dos dados.
* A primeira transformação a ser feita é transformar todas os caracteres para minúsculo e remover espaços extras
* Devido ao grande número de tipos dessas propriedades, não é recomendado utilizar one-hot encoding porque gera esparsidade nos dados.
* Para esses campos, posso fazer as transformações: 
  * Label Encoding
  * Embeddings, se quiser usar redes neurais, também pode funcionar bem com árvores utilizando embeddings pré treinados.
* Problema: temos poucos dados. Precisaríamos utilizar PCA depois de gerar os embeddings para reduzir sua dimensão por conta da maldição da dimensionalidade. Por isso usaremos Label Encoding.

In [10]:
dataset = dataset[dataset["Category"] != "9"].reset_index(drop=True)

In [11]:
dataset.book_title = dataset.book_title.str.lower().str.strip()
dataset.book_author = dataset.book_author.str.lower().str.strip()
dataset.Category = dataset.Category.str.lower().str.strip()

### Visualizações dos Dados e Análise exploratória

precisa analisar o year_of_publication e age e verificar se ele é uma feature relevante ou nao (analisando se o rating muda conforme ele, agrupando em grupos de 5 ou 10 anos, etc), pode deletar elas se for o caso, se for usar precisaremos deixar entre 0 e 1 para ficar tudo na mesma escala


#### Label Encoding

In [12]:
le_book_title = LabelEncoder()
le_book_author = LabelEncoder()
le_category = LabelEncoder()
le_publisher = LabelEncoder()

dataset['book_title_encoded'] = le_book_title.fit_transform(dataset['book_title'])
dataset['book_author_encoded'] = le_book_author.fit_transform(dataset['book_author'])
dataset['category_encoded'] = le_category.fit_transform(dataset['Category'])
dataset['publisher_encoded'] = le_publisher.fit_transform(dataset['publisher'])

Aplicando ``MinMaxScaler`` para garantir que os números estejam entre 0 e 1

In [13]:
sc_book_title = MinMaxScaler()
sc_book_author = MinMaxScaler()
sc_category = MinMaxScaler()
sc_publisher = MinMaxScaler()

dataset['book_title_encoded'] = sc_book_title.fit_transform(dataset[['book_title_encoded']])
dataset['book_author_encoded'] = sc_book_author.fit_transform(dataset[['book_author_encoded']])
dataset['category_encoded'] = sc_category.fit_transform(dataset[['category_encoded']])
dataset['publisher_encoded'] = sc_publisher.fit_transform(dataset[['publisher_encoded']])

#### Salvando os dados

In [14]:
dataset.head()

,rating,age,book_title,book_author,year_of_publication,publisher,Category,book_title_encoded,book_author_encoded,category_encoded,publisher_encoded
0,5,34.7439,the testament,john grisham,1999.0,Dell,['fiction'],0.880062,0.46732,0.54902,0.176
1,10,39.0000,the testament,john grisham,1999.0,Dell,['fiction'],0.880062,0.46732,0.54902,0.176
2,0,34.7439,the testament,john grisham,1999.0,Dell,['fiction'],0.880062,0.46732,0.54902,0.176
3,0,34.7439,the testament,john grisham,1999.0,Dell,['fiction'],0.880062,0.46732,0.54902,0.176
4,8,53.0000,the testament,john grisham,1999.0,Dell,['fiction'],0.880062,0.46732,0.54902,0.176


In [15]:
dataset = dataset[["age", "year_of_publication", "book_title_encoded", "book_author_encoded", "category_encoded", "publisher_encoded", "rating"]]
dataset_x = dataset[["age", "year_of_publication", "book_title_encoded", "book_author_encoded", "category_encoded", "publisher_encoded"]]
dataset_y = dataset[["rating"]]

In [16]:
dataset.head()

,age,year_of_publication,book_title_encoded,book_author_encoded,category_encoded,publisher_encoded,rating
0,34.7439,1999.0,0.880062,0.46732,0.54902,0.176,5
1,39.0000,1999.0,0.880062,0.46732,0.54902,0.176,10
2,34.7439,1999.0,0.880062,0.46732,0.54902,0.176,0
3,34.7439,1999.0,0.880062,0.46732,0.54902,0.176,0
4,53.0000,1999.0,0.880062,0.46732,0.54902,0.176,8


In [17]:
X_train, X_test, y_train, y_test = train_test_split(dataset_x, dataset_y, test_size=0.2, random_state=1)
dados_treinamento = X_train.join(y_train)
dados_teste = X_test.join(y_test)

In [18]:
dataset.to_csv("datasets/dataset.csv", index=False)
dados_teste.to_csv("datasets/dados_teste.csv", index=False)
dados_treinamento.to_csv("datasets/dados_treinamento.csv", index=False)